# 🧠 Mini-GPT
A minimal GPT implementation from scratch. This is designed to run on CPU with limited memory. Only requires `torch`, `numpy`, and `tqdm`.

In [1]:
# Install dependencies (run once)
!pip -q install torch numpy tqdm

In [2]:
# Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import random
import re

In [3]:
# Sample tiny dataset (can be replaced later)
data = """
Once upon a time, there was a small model learning big ideas.
This model understood words and context, not just characters.
It trained every day, dreaming of generalization.
"""

def word_tokenize(text):
    return re.findall(r"\b\w+\b", text.lower())

words = word_tokenize(data)
vocab = sorted(set(words))
vocab_size = len(vocab)
stoi = {w: i for i, w in enumerate(vocab)}
itos = {i: w for w, i in stoi.items()}

def encode(words): return [stoi[w] for w in words]
def decode(tokens): return ' '.join([itos[i] for i in tokens])

In [4]:
# Encode dataset and split
data_ids = torch.tensor(encode(words), dtype=torch.long)
block_size = 8  # number of word tokens per input sequence

def get_batch(batch_size=4):
    ix = torch.randint(len(data_ids) - block_size, (batch_size,))
    x = torch.stack([data_ids[i:i+block_size] for i in ix])
    y = torch.stack([data_ids[i+1:i+block_size+1] for i in ix])
    return x, y

In [5]:
# Single self-attention head
class SelfAttentionHead(nn.Module):
    def __init__(self, n_embed, head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(256, 256)))
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)
        wei = q @ k.transpose(-2, -1) / C**0.5
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        wei = self.dropout(wei)
        v = self.value(x)
        out = wei @ v
        return out

In [6]:
# Multi-Head Attention
class MultiHeadAttention(nn.Module):
    def __init__(self, n_embed, num_heads):
        super().__init__()
        head_size = n_embed // num_heads
        self.heads = nn.ModuleList([
            SelfAttentionHead(n_embed, head_size) for _ in range(num_heads)
        ])
        self.proj = nn.Linear(n_embed, n_embed)  # combine heads
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        return self.dropout(out)

In [7]:
# Feedforward layer
class FeedForward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(0.1),
        )

    def forward(self, x):
        return self.net(x)

In [8]:
# Transformer block
class TransformerBlock(nn.Module):
    def __init__(self, n_embed, num_heads):
        super().__init__()
        self.sa = MultiHeadAttention(n_embed, num_heads)
        self.ff = FeedForward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))  # Self-attention + residual
        x = x + self.ff(self.ln2(x))  # Feedforward + residual
        return x

In [9]:
# Mini-GPT Model
class MiniGPT(nn.Module):
    def __init__(self, vocab_size, n_embed, num_heads=4):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, n_embed)
        self.pos_embedding = nn.Embedding(256, n_embed)
        self.block = TransformerBlock(n_embed, num_heads)
        self.ln_f = nn.LayerNorm(n_embed)  # final layernorm before head
        self.lm_head = nn.Linear(n_embed, vocab_size)

    def forward(self, idx):
        B, T = idx.shape
        tok_emb = self.token_embedding(idx)
        pos_emb = self.pos_embedding(torch.arange(T, device=idx.device))
        x = tok_emb + pos_emb
        x = self.block(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        return logits

In [10]:
# Initialize model
model = MiniGPT(vocab_size=vocab_size, n_embed=64, num_heads=4)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [11]:
# Training loop
for step in tqdm(range(2000)):
    xb, yb = get_batch()
    logits = model(xb)
    B, T, C = logits.shape
    loss = F.cross_entropy(logits.view(B*T, C), yb.view(B*T))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if step % 500 == 0:
        print(f"Step {step}, Loss: {loss.item():.4f}")

  0%|          | 4/2000 [00:00<00:59, 33.43it/s]

Step 0, Loss: 3.4357


 27%|██▋       | 531/2000 [00:08<00:09, 149.22it/s]

Step 500, Loss: 0.0827


 51%|█████▏    | 1028/2000 [00:11<00:06, 157.44it/s]

Step 1000, Loss: 0.0135


 76%|███████▌  | 1511/2000 [00:14<00:04, 110.28it/s]

Step 1500, Loss: 0.0273


100%|██████████| 2000/2000 [00:18<00:00, 108.98it/s]


In [20]:
# Generation function
def generate(model, start_text, max_new_tokens=100):
    model.eval()
    idx = torch.tensor(encode(start_text), dtype=torch.long).unsqueeze(0)
    for _ in range(max_new_tokens):
        logits = model(idx[:, -block_size:])
        next_id = torch.multinomial(F.softmax(logits[:, -1, :], dim=-1), num_samples=1)
        idx = torch.cat((idx, next_id), dim=1)
    return decode(idx[0].tolist())

In [25]:
# Generate text
prompt = "Once upon"
tokens = word_tokenize(prompt)  # Tokenize into words
out = generate(model, tokens, max_new_tokens=26)  # Pass tokens (list of words) to generate

print("Generated:", out)  # Output is already decoded in generate

Generated: once upon a time there was a small model learning big ideas this model understood words and context not just characters it trained every day dreaming of generalization
